# Total Distance Using Haversine Function

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('./filtered-data.csv')
hand = df[df["type"] == "hand"]
ankle = df[df["type"] == "ankle"]
pocket = df[df["type"] == "pocket"]

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # radius of the earth in kilometers
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi / 2) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    d = R * c
    return d * 1000 # distance in meters

def calculateDistance(data):
    df = data.copy()
    df["lat_diff"] = df["lon"].diff()
    df["lon_diff"] = df["lat"].diff()
    df["distance"] = df.apply(lambda x: haversine(x["lat"], x["lon"], x["lat"] + x["lat_diff"], x["lon"] + x["lon_diff"]), axis=1)
    return df

hand = calculateDistance(hand)
ankle = calculateDistance(ankle)
pocket = calculateDistance(pocket)

df = pd.concat([hand, pocket, ankle], ignore_index=True)
df = df[df['distance'].notna()].drop(columns=["lat_diff", "lon_diff"])
df.to_csv("analysis-data.csv", index=False)